In [45]:
# colab 연결용
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda
from torchvision import transforms

In [46]:
import numpy as np

In [94]:
not_norm_transform = transforms.Compose([
    ToTensor(),
    transforms.Lambda(lambda x:x*256)
])

In [137]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=not_norm_transform,
    # transform=ToTensor(),
    # target_transform=Lambda(
    #     lambda y: torch.zeros(10,dtype=torch.float).scatter_(0, torch.tensor(y),value=1)
    # )
)

In [138]:
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [139]:
print(training_data[0][0])
print(training_data[0][0].shape)
print(type(training_data[0][0]))
print(training_data[0][1])

torch.Size([1, 28, 28])
<class 'torch.Tensor'>
9


In [140]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [141]:
from torch import nn

In [142]:
# Feedforward Model
class MLP(torch.nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.linear_relu_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  def forward(self, x):
    outputs = self.linear_relu_stack(x)
    return outputs

In [143]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.001
epochs = 20
model = MLP().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [144]:
for epoch in range(epochs):
  for batch_idx, (X_train, y_train) in enumerate(train_dataloader):
    # Train
    model.train()
    optimizer.zero_grad()
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    pred = model(X_train)
    loss = criterion(pred, y_train)

    loss.backward()
    optimizer.step()
    
    if batch_idx % 100 == 0:
      loss, current = loss.item(), batch_idx * len(X_train)
      print(f"loss : {loss::>7f} [{current:>5d}/{len(train_dataloader.dataset):>5d}]")
      
  test_loss, correct = 0,0
  model.eval()
  with torch.no_grad():
    for X_test,y_test in test_dataloader:
      X_test = X_test.to(device)
      y_test = y_test.to(device)
      pred = model(X_test)
      test_loss += criterion(pred,y_test).item()
      correct += (pred.argmax(1) == y_test).type(torch.float).sum().item()
    test_loss /= len(test_dataloader)
    correct /= len(test_dataloader.dataset)
    print(f"Test Error:\n Accurancy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8}\n")


loss : 2.478134 [    0/60000]
loss : 1.436693 [ 6400/60000]
loss : 1.206418 [12800/60000]
loss : 1.096959 [19200/60000]
loss : 0.941077 [25600/60000]
loss : 0.828868 [32000/60000]
loss : 0.769698 [38400/60000]
loss : 0.583169 [44800/60000]
loss : 0.741031 [51200/60000]
loss : 0.532392 [57600/60000]
Test Error:
 Accurancy: 79.5%, Avg loss: 0.6480090702965315

loss : 0.620497 [    0/60000]
loss : 0.637454 [ 6400/60000]
loss : 0.617838 [12800/60000]
loss : 0.706904 [19200/60000]
loss : 0.609578 [25600/60000]
loss : 0.643370 [32000/60000]
loss : 0.562410 [38400/60000]
loss : 0.563577 [44800/60000]
loss : 0.412550 [51200/60000]
loss : 0.755154 [57600/60000]
Test Error:
 Accurancy: 82.7%, Avg loss: 0.5238133467463796

loss : 0.463059 [    0/60000]
loss : 0.509420 [ 6400/60000]
loss : 0.526924 [12800/60000]
loss : 0.690777 [19200/60000]
loss : 0.560256 [25600/60000]
loss : 0.579279 [32000/60000]
loss : 0.436068 [38400/60000]
loss : 0.397369 [44800/60000]
loss : 0.509776 [51200/60000]
loss : 0

In [ ]:
# Feedforward Model
class NoBatchNormMLP(torch.nn.Module):
  def __init__(self):
    super(NoBatchNormMLP, self).__init__()
    self.linear_relu_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  def forward(self, x):
    outputs = self.linear_relu_stack(x)
    return outputs

In [34]:
no_batch_norm_model = NoBatchNormMLP().to(device)
optimizer = torch.optim.SGD(no_batch_norm_model.parameters(), lr=lr)

In [35]:
for epoch in range(epochs):
  for batch_idx, (X_train, y_train) in enumerate(train_dataloader):
    # Train
    no_batch_norm_model.train()
    optimizer.zero_grad()
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    pred = no_batch_norm_model(X_train)
    loss = criterion(pred, y_train)

    loss.backward()
    optimizer.step()
    
    if batch_idx % 100 == 0:
      loss, current = loss.item(), batch_idx * len(X_train)
      print(f"loss : {loss::>7f} [{current:>5d}/{len(train_dataloader.dataset):>5d}]")
  
  test_loss, correct = 0,0
  no_batch_norm_model.eval()
  with torch.no_grad():
    for X_test,y_test in test_dataloader:
      X_test = X_test.to(device)
      y_test = y_test.to(device)
      pred = no_batch_norm_model(X_test)
      test_loss += criterion(pred,y_test).item()
      correct += (pred.argmax(1) == y_test).type(torch.float).sum().item()
    test_loss /= len(test_dataloader)
    correct /= len(test_dataloader.dataset)
    print(f"Test Error:\n Accurancy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8}\n")

loss : 2.310309 [    0/60000]
loss : 2.305356 [ 6400/60000]
loss : 2.292759 [12800/60000]
loss : 2.310225 [19200/60000]
loss : 2.306816 [25600/60000]
loss : 2.302006 [32000/60000]
loss : 2.307741 [38400/60000]
loss : 2.310967 [44800/60000]
loss : 2.315747 [51200/60000]
loss : 2.304843 [57600/60000]
Test Error:
 Accurancy: 11.0%, Avg loss: 2.305101663443693

loss : 2.303912 [    0/60000]
loss : 2.308083 [ 6400/60000]
loss : 2.297886 [12800/60000]
loss : 2.304216 [19200/60000]
loss : 2.303290 [25600/60000]
loss : 2.311324 [32000/60000]
loss : 2.303388 [38400/60000]
loss : 2.302209 [44800/60000]
loss : 2.307238 [51200/60000]
loss : 2.288414 [57600/60000]
Test Error:
 Accurancy: 11.0%, Avg loss: 2.3050551915624338

loss : 2.294962 [    0/60000]
loss : 2.305453 [ 6400/60000]
loss : 2.315350 [12800/60000]
loss : 2.315060 [19200/60000]
loss : 2.301724 [25600/60000]
loss : 2.318624 [32000/60000]
loss : 2.296133 [38400/60000]
loss : 2.304667 [44800/60000]
loss : 2.304796 [51200/60000]
loss : 2.

In [37]:
from torchvision import transforms

In [68]:
mean = torch.zeros(1)
std = torch.zeros(1)
print('==> Computing mean and std..')
for inputs, _labels in train_dataloader:
  for i in range(1):
    mean[i] += inputs[:,i,:,:].mean()
    std[i] += inputs[:,i,:,:].std()
  mean.div_(len(train_dataloader))
  std.div_(len(train_dataloader))
print(mean, std)


==> Computing mean and std..
tensor([0.0003]) tensor([0.0004])


In [46]:
norm_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [47]:
norm_training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=norm_transform,
    target_transform=Lambda(
        lambda y: torch.zeros(10,dtype=torch.float).scatter_(0, torch.tensor(y),value=1)
    )
)

In [48]:
norm_test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=norm_transform
)

In [49]:
norm_train_dataloader = DataLoader(norm_training_data, batch_size=64, shuffle=True)
norm_test_dataloader = DataLoader(norm_test_data, batch_size=64, shuffle=True)

In [50]:
model_for_norm = MLP().to(device)
optimizer = torch.optim.SGD(model_for_norm.parameters(), lr=lr)

In [51]:
for epoch in range(epochs):
  for batch_idx, (X_train, y_train) in enumerate(norm_train_dataloader):
    # Train
    model_for_norm.train()
    optimizer.zero_grad()
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    pred = model_for_norm(X_train)
    loss = criterion(pred, y_train)

    loss.backward()
    optimizer.step()
    
    if batch_idx % 100 == 0:
      loss, current = loss.item(), batch_idx * len(X_train)
      print(f"loss : {loss::>7f} [{current:>5d}/{len(norm_train_dataloader.dataset):>5d}]")
  
  test_loss, correct = 0,0
  model_for_norm.eval()
  with torch.no_grad():
    for X_test,y_test in norm_test_dataloader:
      X_test = X_test.to(device)
      y_test = y_test.to(device)
      pred = model_for_norm(X_test)
      test_loss += criterion(pred,y_test).item()
      correct += (pred.argmax(1) == y_test).type(torch.float).sum().item()
    test_loss /= len(norm_test_dataloader)
    correct /= len(norm_test_dataloader.dataset)
    print(f"Test Error:\n Accurancy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8}\n")


loss : 2.368590 [    0/60000]
loss : 1.329848 [ 6400/60000]
loss : 1.207564 [12800/60000]
loss : 0.981357 [19200/60000]
loss : 0.874484 [25600/60000]
loss : 1.017581 [32000/60000]
loss : 0.909328 [38400/60000]
loss : 0.761604 [44800/60000]
loss : 0.745831 [51200/60000]
loss : 0.774504 [57600/60000]
Test Error:
 Accurancy: 78.6%, Avg loss: 0.6703777005718012

loss : 0.644553 [    0/60000]
loss : 0.640675 [ 6400/60000]
loss : 0.573136 [12800/60000]
loss : 0.469450 [19200/60000]
loss : 0.618740 [25600/60000]
loss : 0.506961 [32000/60000]
loss : 0.418471 [38400/60000]
loss : 0.502442 [44800/60000]
loss : 0.439745 [51200/60000]
loss : 0.558976 [57600/60000]
Test Error:
 Accurancy: 81.9%, Avg loss: 0.5497547488683349

loss : 0.570003 [    0/60000]
loss : 0.691324 [ 6400/60000]
loss : 0.468275 [12800/60000]
loss : 0.554054 [19200/60000]
loss : 0.697378 [25600/60000]
loss : 0.485486 [32000/60000]
loss : 0.497167 [38400/60000]
loss : 0.534155 [44800/60000]
loss : 0.362863 [51200/60000]
loss : 0

In [145]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.datasets import fetch_openml

In [146]:
mnist = fetch_openml('mnist_784')
x_data = mnist.data.astype('float32')
y_data = mnist.target.astype(int)

In [147]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train.values), torch.tensor(X_test.values)
y_train, y_test = torch.tensor(y_train.values), torch.tensor(y_test.values)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

torch.Size([56000, 784])
torch.Size([14000, 784])
torch.Size([56000])
torch.Size([14000])
